In [17]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from geopy.geocoders import Photon
from geopy.extra.rate_limiter import RateLimiter
import re

In [35]:
df = pd.read_csv('mt20_orre.csv', header=1, sep=';', encoding='latin1', usecols=["Date de début d'appel", "MSISDN Abonné", "Correspondant", "Type de communication", "Durée / Nbr SMS", "Adresse du relais", "IMEI abonné", "IMSI abonné"], dtype= {"MSISDN Abonné": str, "Correspondant": str, "Type de communication": str, "Durée / Nbr SMS": int, "Adresse du relais": str, "IMEI abonné": str, "IMSI abonné": str})
df.head()

,Date de début d'appel,MSISDN Abonné,Correspondant,Type de communication,Durée / Nbr SMS,Adresse du relais,IMEI abonné,IMSI abonné
0,2024-12-05 17:33:49,262692293744,NaN,DATA,0,Ancien Chemin du Phare 97420 Le Port,356415750069700,647000122980517
1,2024-12-05 17:33:49,262692293744,NaN,DATA,81,Ancien Chemin du Phare 97420 Le Port,356415750069700,647000122980517
2,2024-12-05 17:35:10,262692293744,NaN,DATA,18,Rue Salins 97460 Saint Paul,356415750069700,647000122980517
3,2024-12-05 17:35:28,262692293744,NaN,DATA,0,Rue Salins 97460 Saint Paul,356415750069700,647000122980517
4,2024-12-05 17:35:28,262692293744,NaN,DATA,180,Rue Salins 97460 Saint Paul,356415750069700,647000122980517


In [36]:
df = df.rename(columns={"Date de début d'appel": "Date", "MSISDN Abonné": "Abonné", "Type de communication" : "Type d'appel", "Durée / Nbr SMS": "Durée", "Adresse du relais": "Adresse", "IMEI abonné": "IMEI", "IMSI abonné":"IMSI"})

In [37]:
df.head()

,Date,Abonné,Correspondant,Type d'appel,Durée,Adresse,IMEI,IMSI
0,2024-12-05 17:33:49,262692293744,NaN,DATA,0,Ancien Chemin du Phare 97420 Le Port,356415750069700,647000122980517
1,2024-12-05 17:33:49,262692293744,NaN,DATA,81,Ancien Chemin du Phare 97420 Le Port,356415750069700,647000122980517
2,2024-12-05 17:35:10,262692293744,NaN,DATA,18,Rue Salins 97460 Saint Paul,356415750069700,647000122980517
3,2024-12-05 17:35:28,262692293744,NaN,DATA,0,Rue Salins 97460 Saint Paul,356415750069700,647000122980517
4,2024-12-05 17:35:28,262692293744,NaN,DATA,180,Rue Salins 97460 Saint Paul,356415750069700,647000122980517


In [38]:
# Remplacer '0693' par '262693' dans les colonnes 'Abonné' et 'Correspondant'
df['Date'] = pd.to_datetime(df['Date'])
df['Abonné'] = df['Abonné'].replace(r'^0693', '262693', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^0693', '262693', regex=True)
df['Abonné'] = df['Abonné'].replace(r'^0692', '262692', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^0692', '262692', regex=True)
df['Abonné'] = df['Abonné'].replace(r'^06', '336', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^06', '336', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^07', '337', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^02', '2622', regex=True)
# Remplacer les NaN par 'Data'
df['Abonné'] = df['Abonné'].fillna('Data')
df['Correspondant'] = df['Correspondant'].fillna('Data')
df['Abonné'] = df['Abonné'].replace(r'^0692', '262692', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^0692', '262692', regex=True)
df['Abonné'] = df['Abonné'].replace(r'^06', '336', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^06', '336', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^07', '337', regex=True)
df['Correspondant'] = df['Correspondant'].replace(r'^02', '2622', regex=True)
# Remplacer les NaN par 'Data'
df['Abonné'] = df['Abonné'].fillna('Data')
df['Correspondant'] = df['Correspondant'].fillna('Data')
df['Adresse'] = df['Adresse'].str.upper()

df.head()

,Date,Abonné,Correspondant,Type d'appel,Durée,Adresse,IMEI,IMSI
0,2024-12-05 17:33:49,262692293744,Data,DATA,0,ANCIEN CHEMIN DU PHARE 97420 LE PORT,356415750069700,647000122980517
1,2024-12-05 17:33:49,262692293744,Data,DATA,81,ANCIEN CHEMIN DU PHARE 97420 LE PORT,356415750069700,647000122980517
2,2024-12-05 17:35:10,262692293744,Data,DATA,18,RUE SALINS 97460 SAINT PAUL,356415750069700,647000122980517
3,2024-12-05 17:35:28,262692293744,Data,DATA,0,RUE SALINS 97460 SAINT PAUL,356415750069700,647000122980517
4,2024-12-05 17:35:28,262692293744,Data,DATA,180,RUE SALINS 97460 SAINT PAUL,356415750069700,647000122980517


In [39]:
df['Date'] = pd.to_datetime(df['Date'])

# Extraire l'année, le mois et le jour de la semaine
df['Années'] = df['Date'].dt.year
df['Mois'] = df['Date'].dt.month
df['Jour de la semaine'] = df['Date'].dt.day_name()

# Mapper les jours de la semaine en français
jours_semaine_fr = {
    'Monday': 'Lundi',
    'Tuesday': 'Mardi',
    'Wednesday': 'Mercredi',
    'Thursday': 'Jeudi',
    'Friday': 'Vendredi',
    'Saturday': 'Samedi',
    'Sunday': 'Dimanche'
}

# Mapper les mois en français
mois_fr = {
    1: 'Janvier',
    2: 'Février',
    3: 'Mars',
    4: 'Avril',
    5: 'Mai',
    6: 'Juin',
    7: 'Juillet',
    8: 'Août',
    9: 'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}

# Remplacer les numéros de mois par leur équivalent en français
df['Mois'] = df['Mois'].map(mois_fr)
# Remplacer les noms des jours par leur équivalent en français
df['Jour de la semaine'] = df['Jour de la semaine'].map(jours_semaine_fr)

In [40]:
# Fonction pour extraire la ville
def extract_city(address):
    if isinstance(address, str):  # Vérifie si l'adresse est une chaîne
        # Regex pour capturer la ville après le code postal (5 chiffres)
        match = re.search(r'\d{5}\s+(.*)', address)
        if match:
            return match.group(1).strip()  # Retourne la ville sans espaces superflus
    return None  # Retourne None si aucune correspondance n'est trouvée ou si l'adresse n'est pas une chaîne

# Appliquer la fonction pour créer une nouvelle colonne 'Ville'
df['Ville'] = df['Adresse'].apply(extract_city)
df['Ville']= df['Ville'].str.replace("-", " ")
df['Ville']= df['Ville'].str.replace("SAINT", "ST")
df['Ville']= df['Ville'].str.replace("SAINTE", "STE")
df['Ville']= df['Ville'].str.replace("L'", "")
df = df.fillna('')

df.head()

,Date,Abonné,Correspondant,Type d'appel,Durée,Adresse,IMEI,IMSI,Années,Mois,Jour de la semaine,Ville
0,2024-12-05 17:33:49,262692293744,Data,DATA,0,ANCIEN CHEMIN DU PHARE 97420 LE PORT,356415750069700,647000122980517,2024,Décembre,Jeudi,LE PORT
1,2024-12-05 17:33:49,262692293744,Data,DATA,81,ANCIEN CHEMIN DU PHARE 97420 LE PORT,356415750069700,647000122980517,2024,Décembre,Jeudi,LE PORT
2,2024-12-05 17:35:10,262692293744,Data,DATA,18,RUE SALINS 97460 SAINT PAUL,356415750069700,647000122980517,2024,Décembre,Jeudi,ST PAUL
3,2024-12-05 17:35:28,262692293744,Data,DATA,0,RUE SALINS 97460 SAINT PAUL,356415750069700,647000122980517,2024,Décembre,Jeudi,ST PAUL
4,2024-12-05 17:35:28,262692293744,Data,DATA,180,RUE SALINS 97460 SAINT PAUL,356415750069700,647000122980517,2024,Décembre,Jeudi,ST PAUL


In [41]:

# Fonction pour convertir Gauss-Laborde à WGS84
def gauss_laborde_to_wgs84(x, y):
    # Définir le système de projection Gauss-Laborde
    gauss_laborde = Proj(proj='tmerc', lat_0=-21.11666667, lon_0=55.53333333, k=1, x_0=160000, y_0=50000)
    # Définir le système de projection WGS84
    wgs84 = Proj(proj='latlong', datum='WGS84')
    # Créer un transformer pour convertir entre les deux systèmes
    transformer = Transformer.from_proj(gauss_laborde, wgs84)
    # Effectuer la transformation
    lon, lat = transformer.transform(x, y)
    return lat, lon

def count_corr(df):
    # Compter le nombre de contacts par correspondant
    correspondant_counts = df['Correspondant'].value_counts().reset_index()
    # Renommer les colonnes pour plus de clarté
    correspondant_counts.columns = ['Correspondant', 'Nombre de communication']
    # Filtrer pour garder uniquement les correspondants ayant 11 ou 12 caractères
    correspondant_counts = correspondant_counts[correspondant_counts['Correspondant'].str.len().isin([11, 12])]
    # Calculer le nombre total de contacts
    total_contacts = correspondant_counts['Nombre de communication'].sum()
    # Calculer le pourcentage et arrondir à un chiffre après la virgule
    correspondant_counts['Pourcentage'] = ((correspondant_counts['Nombre de communication'] / total_contacts) * 100).round(1)
    # Trier par nombre de contacts du plus élevé au plus bas
    correspondant_counts = correspondant_counts.sort_values(by='Nombre de communication', ascending=False)
    return correspondant_counts

def count_IMEI(df):
    # Compter le nombre de communications par IMEI
    imei_counts = df['IMEI'].value_counts().reset_index()
    # Renommer les colonnes pour plus de clarté
    imei_counts.columns = ['IMEI', 'Nombre de communication']
    # Afficher le DataFrame des IMEI comptabilisés
    return imei_counts

def count_IMSI(df):
    # Compter le nombre de communications par IMEI
    imsi_counts = df['IMSI'].value_counts().reset_index()
    # Renommer les colonnes pour plus de clarté
    imsi_counts.columns = ['IMSI', 'Nombre de communication']
    # Afficher le DataFrame des IMEI comptabilisés
    return imsi_counts

def count_phone_type(df):
    # Compter le nombre de types d'appel
    type_appel_counts = df["Type d'appel"].value_counts().reset_index()
    type_appel_counts.columns = ['Type d\'appel', 'Nombre']
    # Définir les couleurs pour le graphique
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    # Créer le graphique en secteurs
    fig = go.Figure(data=[go.Pie(labels=type_appel_counts['Type d\'appel'],
                                values=type_appel_counts['Nombre'])])
    # Mettre à jour les traces du graphique
    fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                    marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    return fig  # Retourner la figure Plotly

def comm_histo_global(df):
    # Extraire uniquement la date
    df['DateOnly'] = df['Date'].dt.date
    # Compter le nombre de communications par date
    daily_counts = df.groupby('DateOnly').size().reset_index(name='Nombre de communications')
    # Convertir la colonne 'DateOnly' en datetime
    daily_counts['DateOnly'] = pd.to_datetime(daily_counts['DateOnly'])
    # Créer l'histogramme
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=daily_counts['DateOnly'],
        y=daily_counts['Nombre de communications'],
        marker=dict(line=dict(color='black', width=1))  # Bordure noire autour des barres
    ))
    # Mettre à jour la mise en page
    fig.update_layout(
        title='Nombre de communications par jour',
        xaxis_title='Date',
        yaxis_title='Nombre de communications',
        xaxis=dict(tickformat='%Y-%m-%d'),  # Format des dates sur l'axe x
    )
    # Afficher le graphique
    return fig

def comm_histo_monthly(df):
    # Histogramme par mois
    fig_monthly = go.Figure()
    fig_monthly.add_trace(go.Histogram(x=df['Mois'].dropna().astype(str), histfunc='count', name='Communications par mois',marker=dict(line=dict(color='black', width=1))))
    fig_monthly.update_layout(title='Nombre de communications par mois', xaxis_title='Mois', yaxis_title='Nombre de communications')
    return fig_monthly

def comm_histo_weekday(df):
    # Histogramme par jour de la semaine
    # Créer une colonne pour le jour de la semaine
    fig_weekday = go.Figure()
    fig_weekday.add_trace(go.Histogram(x=df['Jour de la semaine'], histfunc='count', name='Communications par jour de la semaine', marker=dict(line=dict(color='black', width=1))))
    fig_weekday.update_layout(title='Nombre de communications par jour de la semaine', xaxis_title='Jour de la semaine', yaxis_title='Nombre de communications')
    return fig_weekday

def comm_histo_hour(df):
    # Convertir la colonne 'Date' en datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # Extraire l'heure de la date
    df['Hour'] = df['Date'].dt.hour
    # Créer l'histogramme par heure
    fig = px.histogram(df, x='Hour', title='Nombre de communications par heure',
                    labels={'Hour': 'Heure', 'count': 'Nombre de communications'},
                    histnorm='',)
    # Ajouter des bordures aux barres
    fig.update_traces(marker=dict(line=dict(color='black', width=1)))
    # Afficher le graphique
    return fig

def adresse_count(df):
    # Compter le nombre de contacts par correspondant
    adresse_counts = df['Adresse'].value_counts().reset_index()
    # Renommer les colonnes pour plus de clarté
    adresse_counts.columns = ['Adresse', 'Nombre de déclenchement']
    # Calculer le nombre total de contacts
    total_contacts = adresse_counts['Nombre de déclenchement'].sum()
    # Calculer le pourcentage et arrondir à un chiffre après la virgule
    adresse_counts['Pourcentage'] = ((adresse_counts['Nombre de déclenchement'] / total_contacts) * 100).round(1)
    # Trier par nombre de contacts du plus élevé au plus bas
    adresse_counts = adresse_counts.sort_values(by='Nombre de déclenchement', ascending=False)
    return adresse_counts

def carto_adresse(df):
    adress_count = adresse_count(df)
    df_merged = adress_count.merge(df, how = 'left', left_on="Adresse", right_on='Adresse')
    # Appliquer la fonction sur les colonnes 'Coordonnée X' et 'Coordonnée Y'
    df_merged[['Latitude', 'Longitude']] = df_merged.apply(lambda row: gauss_laborde_to_wgs84(row['Coordonnée X'], row['Coordonnée Y']), axis=1, result_type='expand')
    fig = px.scatter_map(df_merged, lat="Latitude", lon="Longitude", color="Pourcentage", size="Nombre de déclenchement",
                  color_continuous_scale=px.colors.sequential.Bluered, size_max=15, zoom=10,
                  map_style="carto-positron")
    return fig

def scatter_plot_ville(df):
    fig = px.scatter(df, x= 'Date', y='Ville', color= 'Ville', title='Localisation en fonction de la date')
    return fig

In [42]:
cou = count_corr(df)
cou.head(10)

,Correspondant,Nombre de communication,Pourcentage
4,33773124508,1,33.3
5,447786205094,1,33.3
6,33610783530,1,33.3


In [43]:
cou = count_IMSI(df)
cou

,IMSI,Nombre de communication
0,647000122980517,208


In [44]:
fig = count_phone_type(df)
fig.show()

In [45]:
fig = comm_histo_global(df)
fig.show()

In [46]:
fig = comm_histo_monthly(df)
fig.show()

In [47]:
ad = adresse_count(df)
ad.head()

,Adresse,Nombre de déclenchement,Pourcentage
0,"RÉSIDENCE HÔTELIÈRE ""LE KERVEGUEN"" 84 AV DE LA...",119,57.2
1,"RÉS ""LES BOUCANIERS"" 27 RUE DU BOUCAN CANOT, B...",42,20.2
2,RUE SALINS 97460 SAINT PAUL,9,4.3
3,,8,3.8
4,32 RUE DU GÉNÉRAL LAMBERT RN1 97436 SAINT LEU,5,2.4


In [49]:
geolocator = Photon(user_agent="my_geocoder")

geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Step 4: Define a function to get coordinates
def get_coordinates(address):
    try:
        location = geocode_with_delay(address)
        return (location.latitude, location.longitude) if location else (None, None)
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return (None, None)

# Step 5: Apply the function to the DataFrame
ad['Coordinates'] = ad['Adresse'].apply(get_coordinates)

# Step 6: Split coordinates into separate columns
ad[['Latitude', 'Longitude']] = pd.DataFrame(ad['Coordinates'].tolist(), index=ad.index)

# Display the resulting DataFrame
ad.head(30)

,Adresse,Nombre de déclenchement,Pourcentage,Coordinates,Latitude,Longitude
0,"RÉSIDENCE HÔTELIÈRE ""LE KERVEGUEN"" 84 AV DE LA...",119,57.2,"(-21.0612503, 55.229266)",-21.061250,55.229266
1,"RÉS ""LES BOUCANIERS"" 27 RUE DU BOUCAN CANOT, B...",42,20.2,"(-21.0280041, 55.2264422)",-21.028004,55.226442
2,RUE SALINS 97460 SAINT PAUL,9,4.3,"(-21.014061849999997, 55.27070250583664)",-21.014062,55.270703
3,,8,3.8,"(None, None)",NaN,NaN
4,32 RUE DU GÉNÉRAL LAMBERT RN1 97436 SAINT LEU,5,2.4,"(-21.1630502, 55.2878768)",-21.163050,55.287877
5,107A RUE DE LA BAIE 97460 SAINT PAUL,5,2.4,"(-21.014061849999997, 55.27070250583664)",-21.014062,55.270703
6,"CENTRE COMMERCIAL ""LE FORUM"" 89, RUE DU GÉNÉRA...",5,2.4,"(-21.0529678, 55.2279657)",-21.052968,55.227966
7,CHEMIN CAROSSE VILLÈLE 97460 SAINT PAUL,5,2.4,"(-21.056435450000002, 55.26349225383403)",-21.056435,55.263492
8,RN1 LITTORAL NORD 97426 TROIS BASSINS,3,1.4,"(None, None)",NaN,NaN
9,CHEMIN SAINT PAUL LE BOIS DE NÈFLES PITON 9742...,3,1.4,"(-21.2260641, 55.3011416)",-21.226064,55.301142


In [50]:
import plotly.express as px

fig = px.scatter_map(ad, lat="Latitude", lon="Longitude", color="Pourcentage", size="Nombre de déclenchement",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  map_style="carto-positron")
fig.show()